In [1]:
import os
import numpy as np
import pandas as pd
import itertools as it
from glob import glob
import matplotlib.pyplot as plt
from sbpy.calib import Sun
from astropy import units as u
from astropy.table import Table
from astropy.io import fits

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from astropy.visualization import ImageNormalize, PercentileInterval, AsinhStretch

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp
# from lvmdrp.core.constants import LVM_UNAM_URL
# from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata, fix_lamps_metadata, get_masters_metadata

from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.functions import imageMethod, rssMethod, fluxCalMethod

plt.style.use("seaborn-v0_8-talk")

ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")

In [2]:
frames_table = md.get_metadata(tileid="*", mjd=60147)
frames_table.query("imagetyp == 'bias' | imagetyp == 'dark' | imagetyp == 'flat' | imagetyp == 'arc'", inplace=True)
frames_table = frames_table.loc[frames_table.name.str.contains("sdR")]

[INFO]: loading metadata store at /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60147/raw_metadata.hdf5
[INFO]: found 216 frames in store '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60147/raw_metadata.hdf5'
[INFO]: number of frames after filtering 216
[INFO]: total number of frames found 216


In [3]:
masters_mjd = frames_table.mjd.min()
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")
os.makedirs(masters_path, exist_ok=True)

In [4]:
# bias_analogs = frames_table.query("imagetyp=='bias'").groupby(["mjd", "camera"])

# mbias_paths = []
# for mjd, camera in bias_analogs.groups:
#     biases = bias_analogs.get_group((mjd, camera))
#     print(biases.to_string())
    
#     # let's pre-process the all bias frames
#     pbias_paths = []
#     for bias_frame in biases.to_dict("records"):
#         bias_path = path.full("lvm_raw", camspec=bias_frame["camera"], **bias_frame)
#         pbias_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=bias_frame["imagetyp"], **bias_frame)
#         pbias_paths.append(pbias_path)
        
#         imageMethod.preproc_raw_frame(
#             in_image=bias_path,
#             out_image=pbias_path,
#             display_plots=False
#         )

#     mbias_path = os.path.join(masters_path, f"lvm-mbias-{camera}.fits")
#     mbias_paths.append(mbias_path)
#     imageMethod.create_master_frame(
#         in_images=pbias_paths,
#         out_image=mbias_path,
#         master_mjd=masters_mjd
#     )
# # cache new masters metadata into HDF5 files
# new_masters = md.extract_metadata(frames_paths=mbias_paths)
# md.add_masters(new_masters)

In [5]:
# dark_analogs = frames_table.query("imagetyp=='dark'").groupby(["mjd", "camera"])

# mdark_paths = []
# for mjd, camera in dark_analogs.groups:
#     darks = dark_analogs.get_group((mjd, camera))
#     print(darks.to_string())
    
#     mbias_path = os.path.join(masters_path, f"lvm-mbias-{camera}.fits")
    
#     # let's pre-process the all dark frames
#     ddark_paths = []
#     for dark_frame in darks.to_dict("records"):
#         dark_path = path.full("lvm_raw", camspec=dark_frame["camera"], **dark_frame)
#         pdark_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=dark_frame["imagetyp"], **dark_frame)
#         ddark_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=dark_frame["imagetyp"], **dark_frame)
#         ddark_paths.append(ddark_path)
        
#         imageMethod.preproc_raw_frame(
#             in_image=dark_path,
#             out_image=pdark_path,
#             display_plots=False
#         )
#         imageMethod.detrend_frame(
#             in_image=pdark_path,
#             out_image=ddark_path,
#             in_bias=mbias_path
#         )

#     mdark_path = os.path.join(masters_path, f"lvm-mdark-{camera}.fits")
#     mdark_paths.append(mdark_path)
#     imageMethod.create_master_frame(
#         in_images=ddark_paths,
#         out_image=mdark_path,
#         master_mjd=masters_mjd
#     )
# # cache new masters metadata into HDF5 files
# new_masters = md.extract_metadata(frames_paths=mdark_paths)
# md.add_masters(new_masters)

In [6]:
# flat_analogs = frames_table.query("imagetyp=='flat'").groupby(["camera"])

# mflat_paths = []
# for camera in flat_analogs.groups:
#     flats = flat_analogs.get_group(camera)
    
#     if camera[0] in {"b", "r"}:
#         lamp = "ldls"
#         flats = flats.query("ldls")
#     else:
#         lamp = "quartz"
#         flats = flats.query("quartz")

#     print(flats.to_string())
    
#     mpixmask_path = os.path.join(masters_path, f"lvm-mpixmask-{camera}.fits")
#     mbias_path = os.path.join(masters_path, f"lvm-mbias-{camera}.fits")
#     mdark_path = os.path.join(masters_path, f"lvm-mdark-{camera}.fits")
    
#     # let's pre-process the all flat frames
#     dflat_paths = []
#     for flat_frame in flats.to_dict("records"):
#         flat_path = path.full("lvm_raw", camspec=flat_frame["camera"], **flat_frame)
#         pflat_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=flat_frame["imagetyp"], **flat_frame)
#         dflat_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=flat_frame["imagetyp"], **flat_frame)
#         dflat_paths.append(dflat_path)
        
#         imageMethod.preproc_raw_frame(
#             in_image=flat_path,
#             out_image=pflat_path,
#             in_mask=mpixmask_path,
#             display_plots=False
#         )
#         imageMethod.detrend_frame(
#             in_image=pflat_path,
#             out_image=dflat_path,
#             in_bias=mbias_path,
#             in_dark=mdark_path,
#             in_slitmap=Table(drp.fibermap.data)
#         )

#     mflat_path = os.path.join(masters_path, f"lvm-mflat_{lamp}-{camera}.fits")
#     mflat_paths.append(mflat_path)
#     imageMethod.create_master_frame(
#         in_images=dflat_paths,
#         out_image=mflat_path,
#         master_mjd=masters_mjd
#     )
# # cache new masters metadata into HDF5 files
# new_masters = md.extract_metadata(frames_paths=mflat_paths)
# md.add_masters(new_masters)

In [7]:
arc_analogs = frames_table.query("imagetyp=='arc'").groupby(["camera"])

marc_paths = []
for camera in arc_analogs.groups:
    arcs = arc_analogs.get_group(camera)
    
    if camera[0] == "b":
        lamp = "hgne"
        arcs = arcs.query("camera == @camera & hgne")
    else:
        lamp = "neon"
        arcs = arcs.query("camera == @camera & neon")

    print(arcs.to_string())
    
    mpixmask_path = os.path.join(masters_path, f"lvm-mpixmask-{camera}.fits")
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{camera}.fits")
    mdark_path = os.path.join(masters_path, f"lvm-mdark-{camera}.fits")
    
    # let's pre-process the all arc frames
    darc_paths = []
    for arc_frame in arcs.to_dict("records"):
        arc_path = path.full("lvm_raw", camspec=arc_frame["camera"], **arc_frame)
        parc_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=arc_frame["imagetyp"], **arc_frame)
        darc_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=arc_frame["imagetyp"], **arc_frame)
        darc_paths.append(darc_path)
        
        imageMethod.preproc_raw_frame(
            in_image=arc_path,
            out_image=parc_path,
            in_mask=mpixmask_path,
            display_plots=False
        )
        imageMethod.detrend_frame(
            in_image=parc_path,
            out_image=darc_path,
            in_bias=mbias_path,
            in_dark=mdark_path,
            in_slitmap=Table(drp.fibermap.data)
        )

    marc_path = os.path.join(masters_path, f"lvm-marc_{lamp}-{camera}.fits")
    marc_paths.append(marc_path)
    imageMethod.create_master_frame(
        in_images=darc_paths,
        out_image=marc_path,
        master_mjd=masters_mjd
    )

# cache new masters metadata into HDF5 files
new_masters = md.extract_metadata(frames_paths=marc_paths)
md.add_masters(new_masters)

[INFO]: starting preprocessing of raw image 'sdR-s-b1-00001547.fits.gz'


    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon  hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
0      s    1111  60147  60146      arc  sp1     b1    1547     10.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00001547.fits
59     s    1111  60147  60146      arc  sp1     b1    1546     10.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00001546.fits
87     s    1111  60147  60146      arc  sp1     b1    1548    150.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00001548.fits
110    s    1111  60147  60146      arc  sp1     b1    1549    150.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00001549.fits


[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.71, 2.71, 2.68, 2.69] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 974.00 +/- 1.16 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 982.00 +/- 1.19 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 983.00 +/- 1.32 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 982.00 +/- 1.31 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-b1.fits
[INFO]: building pixel mask
[INFO]: 106029 (0.64 %) pixels masked
[INFO]: replacing 106029 masked pixels wit

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon  hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
8      s    1111  60147  60146      arc  sp2     b2    1549    150.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00001549.fits
88     s    1111  60147  60146      arc  sp2     b2    1547     10.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00001547.fits
153    s    1111  60147  60146      arc  sp2     b2    1548    150.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00001548.fits
158    s    1111  60147  60146      arc  sp2     b2    1546     10.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00001546.fits


[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 150.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.45, 2.46, 2.46, 2.5] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 998.00 +/- 2.23 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 996.00 +/- 2.28 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 1001.00 +/- 2.22 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 999.00 +/- 2.75 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-b2.fits
[INFO]: building pixel mask
[INFO]: 214371 (1.3 %) pixels masked
[INFO]: replacing 214371 masked pixels wit

...

[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.76, 2.65, 2.89, 2.78] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 980.00 +/- 1.11 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 980.00 +/- 1.16 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 990.00 +/- 3.19 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 990.00 +/- 1.24 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z1.fits
[INFO]: building pixel mask
[INFO]: 150150 (0.9 %) pixels masked
[INFO]: replacing 150150 masked pixels with

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime  neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
18     s    1111  60147  60146      arc  sp2     z2    1542     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00001542.fits
27     s    1111  60147  60146      arc  sp2     z2    1544    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00001544.fits
139    s    1111  60147  60146      arc  sp2     z2    1545    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00001545.fits
143    s    1111  60147  60146      arc  sp2     z2    1543     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00001543.fits


[INFO]: starting preprocessing of raw image 'sdR-s-z2-00001542.fits.gz'
[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.6, 2.65, 2.57, 2.72] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 989.00 +/- 1.14 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 983.00 +/- 1.20 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 999.00 +/- 1.18 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 999.00 +/- 1.23 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z2.fits
[INFO]: building pixel mask
[INFO]: 1

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime  neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
41     s    1111  60147  60146      arc  sp3     z3    1542     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00001542.fits
109    s    1111  60147  60146      arc  sp3     z3    1544    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00001544.fits
118    s    1111  60147  60146      arc  sp3     z3    1543     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00001543.fits
122    s    1111  60147  60146      arc  sp3     z3    1545    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00001545.fits


[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.6, 2.65, 2.57, 2.72] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 1009.00 +/- 1.19 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 1013.00 +/- 1.16 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 1013.00 +/- 1.28 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 1011.00 +/- 1.29 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z3.fits
[INFO]: building pixel mask
[INFO]: 60465 (0.36 %) pixels masked
[INFO]: replacing 60465 masked pixels wi